### Nota
Este código importa todos los archivos .csv que esten en la carpeta VentaVehiculosMexico y los agrupa en un solo DF, contiene los datos de Venta de Vehículos en México del 2005-2021. 

In [1]:
# Importing dependencies
import pandas as pd
import glob
import geojson
import geopandas as gpd
import json

In [2]:
# Exports CSV
path1 = "VehiculosExportados" # use your path
all_files1 = glob.glob(path1 + "/*.csv")
li1 = []
for filename in all_files1:
    df = pd.read_csv(filename, index_col=None, header=0)
    li1.append(df)
frame1 = pd.concat(li1, axis=0, ignore_index=True)
countriesTable = pd.read_csv("ExportsMap/tc_pais_destino.csv")
exportsDF = pd.merge(frame1, countriesTable, how="left", left_on="ID_PAIS_DESTINO", right_on="ID_PAIS_DESTINO")
brandTotals = exportsDF.groupby("MARCA")["UNI_VEH"].sum().sort_values(ascending=False)
top = brandTotals[0:7]
years = list(range(2006, 2021))
topExportsDF = exportsDF.loc[exportsDF["MARCA"].isin(top.keys())]
yearsTopExportsDF = topExportsDF.loc[topExportsDF["ANIO"].isin(years)]
exportsCountryYear= yearsTopExportsDF.groupby(["ANIO", "MARCA"])["UNI_VEH"].sum()
exportsCountryYear.to_csv("ExportsMap/brandexports.csv", index=True)

# Sales CSV
path2 = "VentaVehiculosMexico" # use your path
all_files2 = glob.glob(path2 + "/*.csv")
li2 = []
for filename in all_files2:
    df = pd.read_csv(filename, index_col=None, header=0)
    li2.append(df)
frame2 = pd.concat(li2, axis=0, ignore_index=True)
salesDF = pd.merge(frame2, countriesTable, how="left", left_on="ID_PAIS_ORIGEN", right_on="ID_PAIS_DESTINO")
topSalesDF = salesDF.loc[salesDF["MARCA"].isin(top.keys())]
yearsTopSalesDF = topSalesDF.loc[topSalesDF["ANIO"].isin(years)]
salesCountryYear = yearsTopSalesDF.groupby(["ANIO", "MARCA"])["UNI_VEH"].sum()
salesCountryYear.to_csv("ExportsMap/brandsales.csv", index=True)

# Balance CSV
balanceCountryYear = exportsCountryYear - salesCountryYear
balanceCountryYear.to_csv("ExportsMap/brandbalance.csv", index=True)

In [3]:
salesNew = yearsTopSalesDF.groupby(["ANIO", "MARCA"])["UNI_VEH"].sum().to_frame(name='VEHICULOS').reset_index()
exportsNew = yearsTopExportsDF.groupby(["ANIO", "MARCA"])["UNI_VEH"].sum().to_frame(name='VEHICULOS').reset_index()
balanceNew = balanceCountryYear.to_frame(name='VEHICULOS').reset_index()


In [4]:
salesNew

,ANIO,MARCA,VEHICULOS
0,2006,Chrysler,128446
1,2006,Ford Motor,177595
2,2006,General Motors,245090
3,2006,Honda,47471
4,2006,Nissan,228315
...,...,...,...
100,2020,General Motors,150256
101,2020,Honda,47982
102,2020,Nissan,194427
103,2020,Toyota,76577


In [35]:
salesNew.to_json("ExportsMap/brandsales.json", orient="records")
exportsNew.to_json("ExportsMap/brandexports.json", orient="records")
balanceNew.to_json("ExportsMap/brandbalance.json", orient="records")


In [18]:
salesNew.columns

Index(['ANIO', 'MARCA', 'VEHICULOS'], dtype='object')

In [5]:
result = salesNew.to_json(orient="records")
parsed = json.loads(result)
json.dumps(parsed, indent=4)  

'[\n    {\n        "ANIO": 2006,\n        "MARCA": "Chrysler",\n        "VEHICULOS": 128446\n    },\n    {\n        "ANIO": 2006,\n        "MARCA": "Ford Motor",\n        "VEHICULOS": 177595\n    },\n    {\n        "ANIO": 2006,\n        "MARCA": "General Motors",\n        "VEHICULOS": 245090\n    },\n    {\n        "ANIO": 2006,\n        "MARCA": "Honda",\n        "VEHICULOS": 47471\n    },\n    {\n        "ANIO": 2006,\n        "MARCA": "Nissan",\n        "VEHICULOS": 228315\n    },\n    {\n        "ANIO": 2006,\n        "MARCA": "Toyota",\n        "VEHICULOS": 60088\n    },\n    {\n        "ANIO": 2006,\n        "MARCA": "Volkswagen",\n        "VEHICULOS": 135027\n    },\n    {\n        "ANIO": 2007,\n        "MARCA": "Chrysler",\n        "VEHICULOS": 128541\n    },\n    {\n        "ANIO": 2007,\n        "MARCA": "Ford Motor",\n        "VEHICULOS": 147747\n    },\n    {\n        "ANIO": 2007,\n        "MARCA": "General Motors",\n        "VEHICULOS": 230402\n    },\n    {\n        "A